### __Music Recommendation System: That can suggests songs to users based on their listening Patterns__

##### _Author: Muhammad Taimoor Khan_

#### 1. __Importing Libraries__

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm

In [2]:
ntr = 5000
nts = 2000

train_df = pd.read_csv('train.csv',nrows=ntr)

cnames = ['msno','song_id','source_system_tab','source_screen_name','source_type','target']

test_1 = pd.read_csv('train.csv',names=cnames,skiprows=ntr,nrows=nts)
songs_df = pd.read_csv('songs.csv')
members_df = pd.read_csv('members.csv')

#### 2. __Analysing Data__

In [3]:
test_1.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,ipH8Ff6lJyGGZ1ma8358EFbFfMcVFgzqgjCpwZJt+O8=,discover,NaN,top-hits-for-artist,1
1,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,RwXUSDN8lrWLeVWscYtRsAl3P8aPP1ta0r4W5ZAOEDY=,discover,NaN,top-hits-for-artist,1
2,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,9K+br662CDlq+aK9IT/hcQ2aiibdnVnXiekmPQdGT7A=,discover,NaN,top-hits-for-artist,1
3,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,+bXHmkz7OOdkDZOl68QIZA2Xg98fD6bwS6vA+8Mk7oQ=,radio,Radio,radio,1
4,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,5+dF+BW9fEitv98b509WzX0jlqyeBbvDqUrKeMehkC4=,discover,NaN,top-hits-for-artist,1


In [4]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
songs_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [6]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


#### 3. __Data Processing__

In [7]:
test = test_1.drop(['target'],axis=1)
ytr = np.array(test_1['target'])

In [8]:
test.head()

,msno,song_id,source_system_tab,source_screen_name,source_type
0,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,ipH8Ff6lJyGGZ1ma8358EFbFfMcVFgzqgjCpwZJt+O8=,discover,NaN,top-hits-for-artist
1,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,RwXUSDN8lrWLeVWscYtRsAl3P8aPP1ta0r4W5ZAOEDY=,discover,NaN,top-hits-for-artist
2,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,9K+br662CDlq+aK9IT/hcQ2aiibdnVnXiekmPQdGT7A=,discover,NaN,top-hits-for-artist
3,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,+bXHmkz7OOdkDZOl68QIZA2Xg98fD6bwS6vA+8Mk7oQ=,radio,Radio,radio
4,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,5+dF+BW9fEitv98b509WzX0jlqyeBbvDqUrKeMehkC4=,discover,NaN,top-hits-for-artist


In [9]:
#re-arraging the columns to fit the data into code
test_name = ['id','msno','song_id','source_system_tab','source_screen_name','source_type']
test['id'] = np.arange(nts)
test = test[test_name]

In [10]:
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,ipH8Ff6lJyGGZ1ma8358EFbFfMcVFgzqgjCpwZJt+O8=,discover,NaN,top-hits-for-artist
1,1,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,RwXUSDN8lrWLeVWscYtRsAl3P8aPP1ta0r4W5ZAOEDY=,discover,NaN,top-hits-for-artist
2,2,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,9K+br662CDlq+aK9IT/hcQ2aiibdnVnXiekmPQdGT7A=,discover,NaN,top-hits-for-artist
3,3,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,+bXHmkz7OOdkDZOl68QIZA2Xg98fD6bwS6vA+8Mk7oQ=,radio,Radio,radio
4,4,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,5+dF+BW9fEitv98b509WzX0jlqyeBbvDqUrKeMehkC4=,discover,NaN,top-hits-for-artist


In [11]:
songs_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [12]:
print("Data Preprocessing...")

song_cols = ['song_id','artist_name','genre_ids','song_length','language']
train_df = train_df.merge(songs_df[song_cols],on='song_id',how='left')
test = test.merge(songs_df[song_cols],on='song_id',how='left')

Data Preprocessing...


##### 3.1. __Splitting Into Years, Months, and Dates__

In [13]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [14]:
members_df['registration_year'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members_df['registration_month'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members_df['registration_day'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[6:8]))

In [15]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,registration_day
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920,2011,8,20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622,2015,6,28
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712,2016,4,11
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907,2015,9,6
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613,2017,1,26


In [16]:
members_df['expiration_year'] = members_df['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members_df['expiration_month'] = members_df['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members_df['expiration_day'] = members_df['expiration_date'].apply(lambda x: int(str(x)[6:8]))
members_df = members_df.drop(['registration_init_time'],axis=1)

In [17]:
members_df.head()

,msno,city,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20170920,2011,8,20,2017,9,20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20170622,2015,6,28,2017,6,22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20170712,2016,4,11,2017,7,12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150907,2015,9,6,2015,9,7
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170613,2017,1,26,2017,6,13


##### 3.2. __Left Join with Testing and Training Data.__

In [18]:
members_cols = members_df.columns
train_df = train_df.merge(members_df[members_cols], on='msno', how='left')
test = test.merge(members_df[members_cols], on='msno', how='left')


In [19]:
train_df = train_df.fillna(-1)
test = test.fillna(-1)

In [20]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,song_length,language,...,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,206471,52.0,...,0,-1,7,20171005,2012,1,2,2017,10,5
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,284584,52.0,...,24,female,9,20170911,2011,5,25,2017,9,11
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,225396,52.0,...,24,female,9,20170911,2011,5,25,2017,9,11
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,255512,-1.0,...,24,female,9,20170911,2011,5,25,2017,9,11
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,187802,52.0,...,0,-1,7,20171005,2012,1,2,2017,10,5


In [21]:
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type,artist_name,genre_ids,song_length,language,...,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,0,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,ipH8Ff6lJyGGZ1ma8358EFbFfMcVFgzqgjCpwZJt+O8=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),458,244636,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
1,1,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,RwXUSDN8lrWLeVWscYtRsAl3P8aPP1ta0r4W5ZAOEDY=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),465,230086,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
2,2,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,9K+br662CDlq+aK9IT/hcQ2aiibdnVnXiekmPQdGT7A=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),458,278987,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
3,3,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,+bXHmkz7OOdkDZOl68QIZA2Xg98fD6bwS6vA+8Mk7oQ=,radio,Radio,radio,黃美珍 (Jane Huang),458,284525,3.0,...,32,male,9,20171222,2008,2,19,2017,12,22
4,4,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,5+dF+BW9fEitv98b509WzX0jlqyeBbvDqUrKeMehkC4=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),465,263314,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8


##### 3.3 __Collection of Word for Members & Songs__

In [22]:
import gc
del members_df,songs_df; gc.collect()

21

In [23]:
cols = list(train_df.columns)
cols.remove('target')

In [24]:
train_df.head(6)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,song_length,language,...,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,206471,52.0,...,0,-1,7,20171005,2012,1,2,2017,10,5
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,284584,52.0,...,24,female,9,20170911,2011,5,25,2017,9,11
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,225396,52.0,...,24,female,9,20170911,2011,5,25,2017,9,11
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,255512,-1.0,...,24,female,9,20170911,2011,5,25,2017,9,11
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,187802,52.0,...,0,-1,7,20171005,2012,1,2,2017,10,5
5,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,explore,Explore,online-playlist,1,Desiigner,1259,247803,52.0,...,0,-1,7,20171005,2012,1,2,2017,10,5


In [25]:
test.head(6)

,id,msno,song_id,source_system_tab,source_screen_name,source_type,artist_name,genre_ids,song_length,language,...,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,0,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,ipH8Ff6lJyGGZ1ma8358EFbFfMcVFgzqgjCpwZJt+O8=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),458,244636,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
1,1,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,RwXUSDN8lrWLeVWscYtRsAl3P8aPP1ta0r4W5ZAOEDY=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),465,230086,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
2,2,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,9K+br662CDlq+aK9IT/hcQ2aiibdnVnXiekmPQdGT7A=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),458,278987,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
3,3,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,+bXHmkz7OOdkDZOl68QIZA2Xg98fD6bwS6vA+8Mk7oQ=,radio,Radio,radio,黃美珍 (Jane Huang),458,284525,3.0,...,32,male,9,20171222,2008,2,19,2017,12,22
4,4,lUBY3ZofGIMSN5TzcSM/5qF8LVAP3zrZNd6UFTWtZzw=,5+dF+BW9fEitv98b509WzX0jlqyeBbvDqUrKeMehkC4=,discover,-1,top-hits-for-artist,王力宏 (Leehom Wang),465,263314,3.0,...,20,female,4,20170708,2016,2,8,2017,7,8
5,5,nH1t3DQT9gwRxkQ00vQ4tZHLsVkVHiz/FyYOdM3SbbA=,C1wkUNDZyNY1PRCzdvuidiZsLhpQ/mdWX5wRqbSw4bg=,radio,Radio,radio,齊秦,458,266240,3.0,...,32,male,9,20171222,2008,2,19,2017,12,22


##### 3.4 __Label Encoder__

In [27]:
for col in tqdm(cols):
    if train_df[col].dtype == 'object':
        train_df[col] = train_df[col].apply(str)
        test[col] = test[col].apply(str)
        
        label_encoder = LabelEncoder()
        train_df_vals = list(train_df[col].unique())
        test_vals = list(test[col].unique())
        label_encoder.fit(train_df_vals + test_vals)
        train_df[col] = label_encoder.transform(train_df[col])
        test[col] = label_encoder.transform(test[col])
        

100%|██████████| 20/20 [00:00<00:00, 67.59it/s]


##### 3.5 __Song Popularity__

In [28]:
unique_songs = range(max(train_df['song_id'].max(), test['song_id'].max()))
song_pop = pd.DataFrame({'song_id' : unique_songs, 'popularity':0})

train_df_sorted = train_df.sort_values('song_id')
train_df_sorted.reset_index(drop=True,  inplace=True)
test_sorted = test.sort_values('song_id')
test_sorted.reset_index(drop=True, inplace=True)

#### 4. __Model Training & Predictions__

In [29]:
#user library size

X = np.array(train_df.drop(['target'], axis=1))
Y = train_df['target'].values

X_Test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

del train_df, test ; gc.collect()

X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X,Y, test_size=0.3, random_state=12)

del X, Y; gc.collect()

d_train = lgb.Dataset(X_Train, label=Y_Train)
d_Valid = lgb.Dataset(X_Valid, label=Y_Valid)

watchlist = [d_train,d_Valid]


In [35]:
print("Training LGBM Model....")

params = {}

params['learning_rate'] = 0.4
params['application'] = 'binary'
params['max_depth'] = 15
params['num_leaves'] = 2**8
params['verbosity'] = 10
params['metric'] = 'auc'
params['early_stopping_rounds'] = 10

model = lgb.train(params, train_set = d_train, valid_sets=watchlist)


Training LGBM Model....
[LightGBM] [Info] Number of positive: 2788, number of negative: 712
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.775714
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.161514
[LightGBM] [Debug] init for col-wise cost 0.000505 seconds, init for row-wise cost 0.001377 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1485
[LightGBM] [Info] Number of data points in the train set: 3500, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.796571 -> initscore=1.365002
[LightGBM] [Info] Start training from score 1.365002
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 116 and depth = 15
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No f

In [36]:
print("Predictions are being made and saved....")
p_test = model.predict(X_Test)
p_test

Predictions are being made and saved....


array([0.91632169, 0.59138285, 0.93706285, ..., 0.89173393, 0.99013483,
       0.0519424 ])

#### 5. __Creating CSV File to Store ID and Target Columns__

In [37]:
submit = pd.DataFrame()
submit['id'] = ids
submit['target'] = p_test
submit.to_csv('submission.csv.gz', compression='gzip', index=False, float_format= '%.5f')
print("Completed!")

Completed!


In [38]:
yhat = (p_test>0.5).astype(int)
comp = (yhat==ytr).astype(int)
acc = comp.sum()/comp.size*100

print("The accuracy of lgbm model is: {0:f}%".format(acc))

The accuracy of lgbm model is: 78.150000%


In [39]:
rd_seed = np.random.uniform(0,1,nts)
yhat_rand = (rd_seed>0.5).astype(int)
comp_rand = (yhat_rand==ytr).astype(int)
acc_rand = comp_rand.sum()/comp_rand.size*100

print("The accuracy of Random Model is: {0:f}%".format(acc_rand))

The accuracy of Random Model is: 50.650000%


#### __Conclusion__

<code> The Accuracy of LGBM Model = 78.15% </code>

<code> The Accuracy of Random Model = 50.65% </code>

<code> Thus, the LGBM model is better than Random Model for the Test Data.</code>
